In [0]:
#Reset Button. Press when memory goes crazy fat !
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
# imports cell
import numpy as np
import _pickle as cPickle
import sys 
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Deep Learning libs
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Conv2D,Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint


# Let's start with data

We use DeepSig Dataset: RadioML 2016.04C<br>
A synthetic dataset, generated with GNU Radio, consisting of 11 modulations. This is a
variable-SNR dataset with moderate LO drift, light fading, and numerous different
labeled SNR increments for use in measuring performance across different signal and
noise power scenarios.

## Loading data

**The file is formatted as a "pickle" file which can be open for example in python by using cPickle.load(...)**



#### Loading data in Colab

In [24]:
#Loading the data from google drive 

from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive"
filename = "/content/drive/My Drive/RML2016.10b.dat"
open_file = open(filename,'rb')
data = cPickle.load(open_file, encoding='latin1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


#### Loading data locally.

In [0]:
#(Using Jupyter not Colab, Run this cell or above not both)

filename = "RML2016.10b.dat"
open_file = open(filename,'rb')
data = cPickle.load(open_file, encoding='latin1')

In [0]:
# Use lists when accessing data from dict for ease of access.
keys_list = list(data.keys())
temp_list = []
label_list = []
snr_list = []
for i in range(len(keys_list)):
    curr_item = data[keys_list[i]] 
    temp_list.append(curr_item)
    for j in range(curr_item.shape[0]):
        label_list.append(keys_list[i][0])
    #for j in rane(curr_item.shape[0]):
    #    snr_list.append(keys_list[i][1])

# Convert all lists into numpy arrays.
X = np.array(temp_list).reshape(1200000,2,128)
Y = np.array(label_list)

# Clear All un-neccsarry lists created.
temp_list.clear()
label_list.clear()
snr_list.clear()
keys_list.clear()
data.clear()

## Create feature Spaces for data

**Every sample is presented using two vectors each of them has 128 elements. Those two vectors are the in-phase and quadrature phase components of a sample.**

**In this notebook, we consider creating derivatve features and using them in our Neural Net, first alone then in a combination with raw features**

In [0]:
combine_X = np.concatenate((np.apply_along_axis(lambda column:np.gradient(column),2,X),X),axis=1)
X = np.apply_along_axis(lambda column:np.gradient(column),2,X)

## Splitting data

**Split the data into 50% for training/validation and 50% for testing.**

In [0]:
(dtrainX, dtestX, dtrainY, dtestY) = train_test_split(X,Y, test_size=0.50, random_state=42)
(ctrainX, ctestX, ctrainY, ctestY) = train_test_split(combine_X,Y, test_size=0.50, random_state=42)

## Encoding Labels

Our class labels are currently represented as strings; however, Keras will assume that both:

1. Labels are encoded as integers

2. And furthermore, one-hot encoding is performed on these labels making each label represented as a vector rather than an integer

To accomplish this encoding, we can use the LabelBinarizer  class from scikit-learn

In [0]:
lb = LabelBinarizer()
dtrainY = lb.fit_transform(dtrainY)
dtestY = lb.transform(dtestY)
ctrainY = lb.fit_transform(ctrainY)
ctestY = lb.transform(ctestY)

# Defining the Fully Connected Model
<ul>

1. The core data structure of Keras is a model. The simplest type of model is the Sequential model, a linear stack of layers.

2. A 3-layer deep neural network consisiting only of fully connected layers of size 512, 256, 11 neurons.


In [0]:
def build_fc_model(in_shape):
  # Declare layers size
  hidden1_size = 512
  hidden2_size = 256
  hidden3_size = 10

  # Build model
  model = Sequential()
  model.add(Dense(hidden1_size, input_shape = in_shape))
  model.add(Activation('relu'))
  model.add(Dense(hidden2_size))
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(hidden3_size))
  model.add(Activation('softmax'))

  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  return model

### Building the model with input shape. First we build a model for (2,128) input. 

In [30]:
model = build_fc_model((2,128))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 2, 512)            66048     
_________________________________________________________________
activation_4 (Activation)    (None, 2, 512)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 2, 256)            131328    
_________________________________________________________________
activation_5 (Activation)    (None, 2, 256)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total para

## Fitting the model

1. The training process will run for a fixed number of iterations (epochs) through the dataset.

2. We can also set the number of instances that are evaluated before a weight update in the network is performed (batch size).


In [33]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_fc_model_der.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(dtrainX, dtrainY, epochs=150, batch_size=30000, validation_split=0.05,callbacks=[es,mc])

Train on 570000 samples, validate on 30000 samples
Epoch 1/150
570000/570000 [==============================] - 1s 2us/step - loss: 1.4016 - acc: 0.4257 - val_loss: 1.5031 - val_acc: 0.3889

Epoch 00001: val_acc improved from -inf to 0.38887, saving model to best_fc_model_der.h5
Epoch 2/150
570000/570000 [==============================] - 1s 2us/step - loss: 1.4010 - acc: 0.4256 - val_loss: 1.4987 - val_acc: 0.3889

Epoch 00002: val_acc improved from 0.38887 to 0.38893, saving model to best_fc_model_der.h5
Epoch 3/150
570000/570000 [==============================] - 1s 2us/step - loss: 1.4007 - acc: 0.4256 - val_loss: 1.5001 - val_acc: 0.3903

Epoch 00003: val_acc improved from 0.38893 to 0.39030, saving model to best_fc_model_der.h5
Epoch 4/150
570000/570000 [==============================] - 1s 2us/step - loss: 1.4007 - acc: 0.4264 - val_loss: 1.4994 - val_acc: 0.3894

Epoch 00004: val_acc did not improve from 0.39030
Epoch 5/150
570000/570000 [==============================] - 1s 2u

### Building the Fully Connected Model for the first Combination (Raw Features + Derivative Features)


In [42]:
model = build_fc_model((4,128))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 4, 512)            66048     
_________________________________________________________________
activation_16 (Activation)   (None, 4, 512)            0         
_________________________________________________________________
dense_17 (Dense)             (None, 4, 256)            131328    
_________________________________________________________________
activation_17 (Activation)   (None, 4, 256)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                10250     
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
Total para

In [43]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_fc_model_comb1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(ctrainX, ctrainY, epochs=150, batch_size=1000, validation_split=0.05,callbacks=[es,mc])

Train on 570000 samples, validate on 30000 samples
Epoch 1/150
570000/570000 [==============================] - 4s 8us/step - loss: 1.8237 - acc: 0.2887 - val_loss: 1.6516 - val_acc: 0.3476

Epoch 00001: val_acc improved from -inf to 0.34760, saving model to best_fc_model_comb1.h5
Epoch 2/150
570000/570000 [==============================] - 4s 7us/step - loss: 1.5888 - acc: 0.3689 - val_loss: 1.5529 - val_acc: 0.3755

Epoch 00002: val_acc improved from 0.34760 to 0.37553, saving model to best_fc_model_comb1.h5
Epoch 3/150
570000/570000 [==============================] - 4s 7us/step - loss: 1.5273 - acc: 0.3876 - val_loss: 1.5136 - val_acc: 0.3962

Epoch 00003: val_acc improved from 0.37553 to 0.39623, saving model to best_fc_model_comb1.h5
Epoch 4/150
570000/570000 [==============================] - 4s 7us/step - loss: 1.4981 - acc: 0.3981 - val_loss: 1.4939 - val_acc: 0.4006

Epoch 00004: val_acc improved from 0.39623 to 0.40060, saving model to best_fc_model_comb1.h5
Epoch 5/150
5700

# Defining the Convolutional Model
<ul>
    1.  Reshape the input to be the same as the first layer of the network's input

In [0]:
def build_cnn_model(in_shape):
  # Declare layers size
  conv1_kernel_shape=(3,1)
  conv1_number_of_filters=64
  conv2_kernel_shape=(3,2)
  conv2_number_of_filters=16
  dense1_size = 128
  dense2_size = 10


  # Build model
  model_conv = Sequential()
  model_conv.add(Reshape((128,in_shape[0],1), input_shape=in_shape))
  model_conv.add(Conv2D(conv1_number_of_filters, conv1_kernel_shape, strides=1,
                   padding='same', data_format='channels_last',activation='relu'))
  model_conv.add(Conv2D(conv2_number_of_filters, conv2_kernel_shape, strides=1,
                   padding='same', data_format='channels_last',activation='relu'))
  model_conv.add(Flatten())
  model_conv.add(Dense(dense1_size, activation='relu'))
  model_conv.add(Dense(dense2_size, activation='softmax'))

  # Compile model
  model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model_conv.summary()
  return model_conv

### Building the model with input shape. First we build a model for (2,128) input. 

In [56]:
model = build_cnn_model((2,128))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 128, 2, 1)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 2, 64)        256       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 128, 2, 16)        6160      
_________________________________________________________________
flatten_11 (Flatten)         (None, 4096)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               524416    
_________________________________________________________________
dense_28 (Dense)             (None, 10)                1290      
Total params: 532,122
Trainable params: 532,122
Non-trainable params: 0
_________________________________________________________________


In [57]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_cnn_model_der.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(dtrainX, dtrainY, epochs=150, batch_size=1000, validation_split=0.05,callbacks=[es,mc])

Train on 570000 samples, validate on 30000 samples
Epoch 1/150
570000/570000 [==============================] - 13s 23us/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3025 - val_acc: 0.1038

Epoch 00001: val_acc improved from -inf to 0.10383, saving model to best_cnn_model_der.h5
Epoch 2/150
570000/570000 [==============================] - 10s 18us/step - loss: 2.3021 - acc: 0.1060 - val_loss: 2.3013 - val_acc: 0.1184

Epoch 00002: val_acc improved from 0.10383 to 0.11837, saving model to best_cnn_model_der.h5
Epoch 3/150
570000/570000 [==============================] - 10s 18us/step - loss: 2.2995 - acc: 0.1152 - val_loss: 2.2973 - val_acc: 0.1196

Epoch 00003: val_acc improved from 0.11837 to 0.11957, saving model to best_cnn_model_der.h5
Epoch 4/150
570000/570000 [==============================] - 10s 18us/step - loss: 2.2946 - acc: 0.1197 - val_loss: 2.2929 - val_acc: 0.1185

Epoch 00004: val_acc did not improve from 0.11957
Epoch 5/150
570000/570000 [============================

KeyboardInterrupt: ignored

### Defining the CNN Model on the combined data of raw features and derivative features (Same but different input shape)


In [62]:
model = build_cnn_model((4,128))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_7 (Reshape)          (None, 128, 4, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 128, 4, 64)        256       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 128, 4, 16)        6160      
_________________________________________________________________
flatten_13 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               1048704   
_________________________________________________________________
dense_32 (Dense)             (None, 10)                1290      
Total params: 1,056,410
Trainable params: 1,056,410
Non-trainable params: 0
_________________________________________________________________


In [63]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_cnn_model_comb1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(ctrainX, ctrainY, epochs=150, batch_size=1000, validation_split=0.05,callbacks=[mc])

Train on 570000 samples, validate on 30000 samples
Epoch 1/150
570000/570000 [==============================] - 21s 37us/step - loss: 1.8366 - acc: 0.2797 - val_loss: 1.5759 - val_acc: 0.3644

Epoch 00001: val_acc improved from -inf to 0.36440, saving model to best_cnn_model_comb1.h5
Epoch 2/150
570000/570000 [==============================] - 20s 36us/step - loss: 1.4605 - acc: 0.4155 - val_loss: 1.3872 - val_acc: 0.4363

Epoch 00002: val_acc improved from 0.36440 to 0.43627, saving model to best_cnn_model_comb1.h5
Epoch 3/150
570000/570000 [==============================] - 20s 35us/step - loss: 1.3645 - acc: 0.4499 - val_loss: 1.3464 - val_acc: 0.4517

Epoch 00003: val_acc improved from 0.43627 to 0.45167, saving model to best_cnn_model_comb1.h5
Epoch 4/150
570000/570000 [==============================] - 20s 34us/step - loss: 1.3249 - acc: 0.4639 - val_loss: 1.3252 - val_acc: 0.4655

Epoch 00004: val_acc improved from 0.45167 to 0.46550, saving model to best_cnn_model_comb1.h5
Epoc

KeyboardInterrupt: ignored